In [2]:
import pandas as pd
import openpyxl
import awswrangler as awr

xlsx = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas\placas_desativadas.xlsx"
df_outras = pd.read_excel(xlsx, engine = 'openpyxl')


In [3]:
df_outras.shape[0]

77

In [4]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes\sql\others_query.sql"
with open(query_path, "r") as file:
    query = file.read()

df_query = awr.athena.read_sql_query(query, database='silver')
df_query.head()  

,placa,chassi,id_placa,id_veiculo,id_carroceria,matricula,conjunto,unidade,consultor,status,...,empresa,coverage,beneficio,categoria,tipo_categoria,data_ativacao_beneficio,status_beneficio,data_atualizacao,consultor_ativo,id_beneficio
0,REA0B58,9BM958443MB192917,339427,339427,<NA>,1518,1819,MICRO B - MARIA ISABELA CARNEIRO DE OLIVEIRA,Maria Isabela Carneiro O,ATIVO,...,Tag,698582,Casco PT (VEÍCULO),PESADOS - CAVALO MECÂNICO,MERCEDES,2025-08-26,ATIVO,2025-08-26,S,34
1,HFT5A79,9BVJM30C7BE777523,308939,308939,<NA>,1519,1810,TRANSDESK DIGITAL LTDA,Julia Badia,ATIVO,...,Tag,698535,APP,PADRÃO,PADRÃO,2025-09-01,ATIVO,2025-09-01,S,48
2,HFT5A79,9BVJM30C7BE777523,308939,308939,<NA>,1519,1810,TRANSDESK DIGITAL LTDA,Julia Badia,ATIVO,...,Tag,698529,Casco PT (VEÍCULO),PESADOS - CAVALO MECÂNICO,VOLVO,2025-09-01,ATIVO,2025-09-01,S,34
3,AUF0D91,9BVAS02C7BE775482,290791,290791,<NA>,1521,1817,UNIDADE ITAPEJARA,Milena Dalbosco,ATIVO,...,Tag,698576,APP,PADRÃO,PADRÃO,2025-08-26,ATIVO,2025-08-26,S,48
4,AUF0D91,9BVAS02C7BE775482,290791,290791,<NA>,1521,1817,UNIDADE ITAPEJARA,Milena Dalbosco,ATIVO,...,Tag,698575,RCF,PESADOS - CAVALO MECÂNICO,PADRÃO,2025-08-26,ATIVO,2025-08-26,S,40


In [5]:
# Vamos criar uma nova coluna em df_outras chamada 'status_beneficio_query', puxando o status correspondente da df_query pela chave 'chassi'

# Primeiro, garantir que ambas as colunas 'chassi' estejam normalizadas (string, strip, upper)
df_outras['chassi'] = df_outras['chassi'].astype(str).str.strip().str.upper()
df_query['chassi'] = df_query['chassi'].astype(str).str.strip().str.upper()

# Criar um dicionário de mapeamento chassi -> status_beneficio (pegando o primeiro status encontrado para cada chassi)
status_map = df_query.drop_duplicates('chassi').set_index('chassi')['status_beneficio'].to_dict()

# Adicionar a nova coluna em df_outras
df_outras['status_beneficio_query'] = df_outras['chassi'].map(status_map)


In [6]:
df_outras.shape[0]

77

In [7]:
df_outras['status_beneficio_query'].value_counts()
#NOVO significa aguardando ativação

status_beneficio_query
CANCELADO    50
NOVO         16
ATIVO         8
RENOVAÇÃO     2
Name: count, dtype: int64

In [8]:
df_outras_naocancelados = df_outras[df_outras['status_beneficio_query'] != "CANCELADO"]
# Para ver o número de linhas:
print(df_outras_naocancelados.shape[0])
# Para ver os valores únicos de status_beneficio_query:
print(df_outras_naocancelados['status_beneficio_query'].value_counts())

27
status_beneficio_query
NOVO         16
ATIVO         8
RENOVAÇÃO     2
Name: count, dtype: int64


In [9]:
df_outras.shape[0]

77

In [10]:
df_outras[df_outras['status_beneficio_query']=="CANCELADO"].shape[0]

50